In [ ]:
# !pip install pyLDAvis

In [ ]:
from wordcloud import WordCloud
import gensim
from gensim.models import CoherenceModel
import pickle
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=605654cee988ee1b28f5e215f128c77e905e2cd5e7b6b01bd8d17a041bd1ec4c
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [ ]:
from data_cleaning import Data_Cleaning
from data_preprocessing import Data_Preprocessing

importing Jupyter notebook from data_cleaning.ipynb
importing Jupyter notebook from data_preprocessing.ipynb


<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence \s
<input>:16: DeprecationWarning: invalid escape sequence 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# function to train the lda models
def run_lda_model(dictt, corp, num_top, ran_st, up_every, ch_size, pas):
  model = gensim.models.ldamodel.LdaModel(corpus=corp,
                                           id2word=dictt,
                                           num_topics=num_top,
                                           random_state=ran_st,
                                           update_every=up_every,
                                           chunksize=ch_size,
                                           passes=pas,
                                           alpha='auto',
                                           per_word_topics=True)
  return model

In [ ]:
# plot the most common 1000 word in the given dataset
def get_most_common_words(texts):
  text = ""
  for txt in texts:
    text += txt

  wordcloud = WordCloud(background_color="black", max_words=1000, contour_width=5, contour_color='steelblue')
  wordcloud.generate(text)
  return wordcloud.to_image()

In [ ]:
# return the topic from model in a format of topic_index, topic_string
# in this case it is more easier to read the topics
# topic is 10 words
def show_model_topics(model):
  topics = []
  for topic_id in range(model.num_topics):
    topic = model.show_topic(topic_id, 10)
    topic_words = [ w for w, _ in topic ]
    print('{}: {}'.format(topic_id, ' '.join(topic_words)))
    topics.append((topic_id, ' '.join(topic_words)))
  return topics

In [ ]:
# measure the model performance using perplexity metric
# the lower perplexity the better model
def compute_perplexity(model, corpus):
  model_perplexity = model.log_perplexity(corpus)
  return model_perplexity

In [ ]:
# measure the model performance using coherence score
#the higher the better
def computer_coherence_score(model, lemmtize_text, dictionary, coherence='c_v'):
  coherence_model = CoherenceModel(model=model, texts=lemmtize_text, dictionary=dictionary, coherence=coherence)
  coherence_score = coherence_model.get_coherence()
  return coherence_score

In [ ]:
# save the lda model
def save_model(model, file_name):
  pickle.dump(model, open(file_name, 'wb'))
  print("Model saved...")
  return

In [ ]:
# load lda model 
def load_model(file_name):
  model = pickle.load(open(file_name, 'rb'))
  return model

In [ ]:
# visualize topics
def visualize_topics(model, dictionary, corpus):
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
  return vis

In [ ]:
#function to retrive the best topics number based on models coherence scores
def get_optimal_topics_num(scores):
  max_value =max(scores)
  indx = scores.index(max_value)
  return max_value, indx

In [ ]:
# plot a graph to visualize the number of topics for each tried model
def plot_num_topics(scores, indx, x):
  col =[] # hold different color to make highest point with unique color
  for i in range(len(x)):
    if (i== indx):
      col.append('red')
    else:
      col.append('blue')

  plt.plot(x, scores, c="black")
  for i in range(len(x)):
    plt.scatter(x[i], scores[i], c = col[i], s = 10,
          linewidth = 5)
  
  plt.xlabel('number of topics')
  plt.ylabel('coherence score')
  plt.title('optimal # topics')
  plt.show()

In [ ]:
# train different models with hyperparamtr tuning and calculate their coherence scores to get the best model among.
def compute_coherence_scores(dictionary, corpus, lemma_text, limit, start=2, step=1):
  x = x = range(start, limit, step)
  coherence_scores = []
  models = []
  for num_topics in range(start, limit, step):
      model = run_lda_model(dictionary, corpus, num_topics, 24, 1, 500, 15)
      models.append(model)
      coherence_score = computer_coherence_score(model, lemma_text, dictionary, 'c_v')
      coherence_scores.append(coherence_score)

  return models, coherence_scores, x